In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [5]:
import pandas as pd
import numpy as np

## Convert to sparse matrix
from scipy import sparse
import scipy.sparse as sp

# Load Path

In [6]:
## Load Path
# get_path = 'drive/MyDrive/Skip-GNN/'
data_path = 'drive/MyDrive/Skip-GNN/data/IAV/'

In [7]:
protein_list = pd.read_csv(data_path + 'protein_list.csv')
# protein_list = pd.read_csv('drive/MyDrive/Skip-GNN/data/IAV/protein_list.csv')
protein_list ## 15685 rows

,Unnamed: 0,Protein1_ID
0,0,P03428
1,1,P03431
2,2,P03433
3,3,P03452
4,4,P03466
...,...,...
15680,15680,Q6NUS8
15681,15681,P12018
15682,15682,Q96IU2
15683,15683,Q6PEW1


# SET EMBEDDING METHOD

In [8]:
### CHANGE EMBEDDING METHOD HERE
# g_emb_mtd = 's2v-grarep-sdne'
g_emb_mtd = 's2v-sdne'
p_emb_mtd = 'DPC'

# Pre-process Protein Embedding

iFeature (https://github.com/Superzchen/iFeature), Input: protein sequence file in fasta format

In [10]:
p_data_path = data_path + 'protein_embeddings/'
p_emb = pd.read_csv(p_data_path + p_emb_mtd + '.tsv', sep='\t')
p_emb

,#,AA,AC,AD,AE,AF,AG,AH,AI,AK,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,P03428,0.006596,0.000000,0.003958,0.001319,0.000000,0.005277,0.000000,0.006596,0.003958,...,0.001319,0.002639,0.002639,0.001319,0.000000,0.003958,0.002639,0.000000,0.000000,0.000000
1,P03431,0.001323,0.000000,0.001323,0.002646,0.001323,0.002646,0.001323,0.002646,0.002646,...,0.002646,0.001323,0.000000,0.002646,0.006614,0.002646,0.005291,0.000000,0.001323,0.000000
2,P03433,0.002797,0.001399,0.002797,0.006993,0.000000,0.000000,0.000000,0.005594,0.001399,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001399,0.001399,0.002797,0.000000,0.001399
3,P03452,0.007092,0.001773,0.005319,0.003546,0.001773,0.005319,0.000000,0.008865,0.003546,...,0.000000,0.003546,0.005319,0.005319,0.001773,0.003546,0.000000,0.005319,0.001773,0.001773
4,P03466,0.012072,0.004024,0.000000,0.004024,0.006036,0.008048,0.004024,0.002012,0.000000,...,0.000000,0.000000,0.000000,0.002012,0.002012,0.004024,0.000000,0.000000,0.002012,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15680,I6T1Z7,0.006596,0.000000,0.003958,0.001319,0.000000,0.003958,0.000000,0.006596,0.003958,...,0.001319,0.002639,0.002639,0.001319,0.000000,0.003958,0.002639,0.000000,0.000000,0.000000
15681,Q67157,0.003984,0.000000,0.003984,0.007968,0.003984,0.015936,0.000000,0.003984,0.007968,...,0.000000,0.003984,0.000000,0.003984,0.003984,0.003984,0.000000,0.003984,0.000000,0.000000
15682,I6TAH8,0.012072,0.004024,0.000000,0.006036,0.008048,0.008048,0.004024,0.002012,0.002012,...,0.000000,0.000000,0.000000,0.002012,0.000000,0.004024,0.000000,0.000000,0.002012,0.000000
15683,Q02600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
## Map to Index
p_emb = p_emb.rename(columns={'#':'Protein1_ID'})
p_emb_f = pd.merge(protein_list, p_emb, on=['Protein1_ID'])
p_emb_f = p_emb_f.drop(columns=['Protein1_ID'])
p_emb_f

,Unnamed: 0,AA,AC,AD,AE,AF,AG,AH,AI,AK,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
0,0,0.006596,0.000000,0.003958,0.001319,0.000000,0.005277,0.000000,0.006596,0.003958,...,0.001319,0.002639,0.002639,0.001319,0.000000,0.003958,0.002639,0.000000,0.000000,0.000000
1,1,0.001323,0.000000,0.001323,0.002646,0.001323,0.002646,0.001323,0.002646,0.002646,...,0.002646,0.001323,0.000000,0.002646,0.006614,0.002646,0.005291,0.000000,0.001323,0.000000
2,2,0.002797,0.001399,0.002797,0.006993,0.000000,0.000000,0.000000,0.005594,0.001399,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001399,0.001399,0.002797,0.000000,0.001399
3,3,0.007092,0.001773,0.005319,0.003546,0.001773,0.005319,0.000000,0.008865,0.003546,...,0.000000,0.003546,0.005319,0.005319,0.001773,0.003546,0.000000,0.005319,0.001773,0.001773
4,4,0.012072,0.004024,0.000000,0.004024,0.006036,0.008048,0.004024,0.002012,0.000000,...,0.000000,0.000000,0.000000,0.002012,0.002012,0.004024,0.000000,0.000000,0.002012,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15680,15680,0.003831,0.000000,0.001916,0.003831,0.011494,0.001916,0.003831,0.003831,0.003831,...,0.000000,0.000000,0.000000,0.001916,0.000000,0.000000,0.000000,0.001916,0.000000,0.000000
15681,15681,0.006944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.006944,0.000000,0.006944,0.000000,0.000000,0.006944,0.006944
15682,15682,0.034335,0.004292,0.000000,0.004292,0.000000,0.008584,0.004292,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004292,0.000000,0.000000,0.000000,0.000000
15683,15683,0.000000,0.000000,0.007481,0.002494,0.004988,0.002494,0.004988,0.002494,0.000000,...,0.000000,0.002494,0.000000,0.000000,0.000000,0.000000,0.000000,0.002494,0.000000,0.000000


In [ ]:
p_data_path = data_path + 'protein_embeddings/'
p_emb = pd.read_csv(p_data_path + p_emb_mtd + '.tsv', sep='\t')

## Map to Index
p_emb = p_emb.rename(columns={'#':'Protein1_ID'})
p_emb_f = pd.merge(protein_list, p_emb, on=['Protein1_ID'])
p_emb_f = p_emb_f.drop(columns=['Protein1_ID'])

## Format dataframe
df1 = pd.DataFrame.from_records(p_emb_f.values)
idx_col = df1.iloc[:, 0].astype(int)
df1 = df1.set_index(idx_col).drop(columns=[0])

In [ ]:
df1

,1,2,3,4,5,6,7,8,9,10,...,391,392,393,394,395,396,397,398,399,400
0,,,,,,,,,,,,,,,,,,,,,
0,0.006596,0.000000,0.003958,0.001319,0.000000,0.005277,0.000000,0.006596,0.003958,0.002639,...,0.001319,0.002639,0.002639,0.001319,0.000000,0.003958,0.002639,0.000000,0.000000,0.000000
1,0.001323,0.000000,0.001323,0.002646,0.001323,0.002646,0.001323,0.002646,0.002646,0.005291,...,0.002646,0.001323,0.000000,0.002646,0.006614,0.002646,0.005291,0.000000,0.001323,0.000000
2,0.002797,0.001399,0.002797,0.006993,0.000000,0.000000,0.000000,0.005594,0.001399,0.008392,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001399,0.001399,0.002797,0.000000,0.001399
3,0.007092,0.001773,0.005319,0.003546,0.001773,0.005319,0.000000,0.008865,0.003546,0.003546,...,0.000000,0.003546,0.005319,0.005319,0.001773,0.003546,0.000000,0.005319,0.001773,0.001773
4,0.012072,0.004024,0.000000,0.004024,0.006036,0.008048,0.004024,0.002012,0.000000,0.004024,...,0.000000,0.000000,0.000000,0.002012,0.002012,0.004024,0.000000,0.000000,0.002012,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15680,0.003831,0.000000,0.001916,0.003831,0.011494,0.001916,0.003831,0.003831,0.003831,0.003831,...,0.000000,0.000000,0.000000,0.001916,0.000000,0.000000,0.000000,0.001916,0.000000,0.000000
15681,0.006944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006944,...,0.000000,0.000000,0.000000,0.006944,0.000000,0.006944,0.000000,0.000000,0.006944,0.006944
15682,0.034335,0.004292,0.000000,0.004292,0.000000,0.008584,0.004292,0.000000,0.000000,0.025751,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004292,0.000000,0.000000,0.000000,0.000000


In [ ]:
p_features = df1.sort_index().values
# print(p_features[0])
print(p_features.shape)

(15685, 400)


In [ ]:
p_features

array([[0.00659631, 0.        , 0.00395778, ..., 0.        , 0.        ,
        0.        ],
       [0.00132275, 0.        , 0.00132275, ..., 0.        , 0.00132275,
        0.        ],
       [0.0027972 , 0.0013986 , 0.0027972 , ..., 0.0027972 , 0.        ,
        0.0013986 ],
       ...,
       [0.03433476, 0.00429185, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0074813 , ..., 0.00249377, 0.        ,
        0.        ],
       [0.00497512, 0.00248756, 0.00497512, ..., 0.00248756, 0.        ,
        0.        ]])

In [ ]:
## Save p_features
p_feat_sparse = sp.csr_matrix(p_features, dtype=float)
print(p_feat_sparse)

  (0, 0)	0.0065963060686015
  (0, 2)	0.0039577836411609
  (0, 3)	0.0013192612137203
  (0, 5)	0.0052770448548812
  (0, 7)	0.0065963060686015
  (0, 8)	0.0039577836411609
  (0, 9)	0.0026385224274406
  (0, 10)	0.0039577836411609
  (0, 11)	0.0026385224274406
  (0, 12)	0.0013192612137203
  (0, 13)	0.0026385224274406
  (0, 14)	0.0026385224274406
  (0, 15)	0.0013192612137203
  (0, 16)	0.0052770448548812
  (0, 17)	0.0065963060686015
  (0, 19)	0.0013192612137203
  (0, 26)	0.0013192612137203
  (0, 28)	0.0026385224274406
  (0, 30)	0.0013192612137203
  (0, 38)	0.0013192612137203
  (0, 40)	0.0039577836411609
  (0, 41)	0.0026385224274406
  (0, 42)	0.0013192612137203
  (0, 43)	0.0039577836411609
  (0, 46)	0.0013192612137203
  :	:
  (15684, 335)	0.0024875621890547
  (15684, 336)	0.0024875621890547
  (15684, 340)	0.0049751243781094
  (15684, 344)	0.0049751243781094
  (15684, 345)	0.0024875621890547
  (15684, 347)	0.0099502487562189
  (15684, 348)	0.0024875621890547
  (15684, 349)	0.0049751243781094
  (1

In [ ]:
p_save_path = data_path + 'protein_embeddings/'
sparse.save_npz(p_save_path + p_emb_mtd +'.npz', p_feat_sparse)

# Pre-process Graph Embedding

https://github.com/shenweichen/GraphEmbedding

https://github.com/xiangyue9607/BioNEV

In [ ]:
# ## Pre-process graph emb
# g_data_path = 'drive/MyDrive/Skip-GNN/data/IAV/graph_embeddings/'
# emb = pd.read_csv(g_data_path + g_emb_mtd + '.csv', skiprows=1, header = None).sort_values(by = [0]).set_index([0]) 
# emb ## 15685 rows
# print(emb.shape)

# ## Use emb.loc[index] to locate row using index, column[0]

# ## setdiff1d: 1D array of values in ar1 that are not in ar2
# for i in np.setdiff1d(np.arange(len(protein_list)), emb.index.values): 
    
#     ## manually insert emb for protein indexes with no node2vec node attribute embedding 
#     emb.loc[i] = (np.sum(emb.values, axis = 0)/emb.values.shape[0]) ## axis = 0 (column-wise sum), emb.values.shape = (len(protein_list), dim of graph embedding (e.g. node2vec=128))

# g_features = emb.sort_index().values
# print(g_features.shape)

# *** Read Embeddings ***

## Read Protein Embedding

In [ ]:
p_emb_path = 'drive/MyDrive/Skip-GNN/data/IAV/protein_embeddings/'

read_p_emb = sparse.load_npz(p_emb_path + p_emb_mtd + '.npz')
p_features = read_p_emb.toarray()
print(p_features)
print(p_features.shape)

## Read Ensemble Graph Embeddings

In [ ]:
g_emb_path = 'drive/MyDrive/Skip-GNN/data/IAV/optim_graph_emb/'

read_g_emb = sparse.load_npz(g_emb_path + g_emb_mtd + '.npz')
g_features=read_g_emb.toarray()
print(g_features)
print(g_features.shape)

## Concat protein emb with graph emb

In [ ]:
### Concatenate
concat_emb_final = np.concatenate((p_features, g_features), axis = 1)
print(concat_emb_final)
print(concat_emb_final.shape)

In [ ]:
adj_mat_sparse = sp.csr_matrix(concat_emb_final, dtype=float)
print(adj_mat_sparse)

In [ ]:
save_to_path = data_path + 'concat_embeddings/'
sparse.save_npz(save_to_path + p_emb_mtd + '_' + g_emb_mtd +'.npz', adj_mat_sparse)

### Normalize

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
norm_concat_emb = normalize(concat_emb_final, copy=False)

In [ ]:
adj_mat_sparse = sp.csr_matrix(norm_concat_emb, dtype=float)
print(adj_mat_sparse)

In [ ]:
save_to_path = data_path + 'concat_embeddings/'
sparse.save_npz(save_to_path + p_emb_mtd + '_' + g_emb_mtd +'_norm.npz', adj_mat_sparse)

# C/T/D

In [ ]:
def func_read_p_emb(p_emb_mtd):
  p_data_path = data_path + 'protein_embeddings/'
  p_emb = pd.read_csv(p_data_path + p_emb_mtd + '.tsv', sep='\t')

  ## Map to Index
  p_emb = p_emb.rename(columns={'#':'Protein1_ID'})
  p_emb_f = pd.merge(protein_list, p_emb, on=['Protein1_ID'])
  p_emb_f = p_emb_f.drop(columns=['Protein1_ID'])

  ## Format dataframe
  df1 = pd.DataFrame.from_records(p_emb_f.values)
  idx_col = df1.iloc[:, 0].astype(int)
  df1 = df1.set_index(idx_col).drop(columns=[0])

  # print(p_emb_mtd)
  # print(df1)

  p_emb_feat = df1.sort_index().values
  print(p_emb_feat)
  print(p_emb_feat.shape)

  p_save_path = data_path + 'protein_embeddings/'
  sparse.save_npz(p_save_path + p_emb_mtd + '.npz', ctd_sparse)

  return p_emb_feat

In [ ]:
c_emb = func_read_p_emb('CTDC')
t_emb = func_read_p_emb('CTDT')
d_emb = func_read_p_emb('CTDD')

In [ ]:
concat_ctd = np.concatenate([c_emb, t_emb, d_emb], axis = 1)
concat_ctd

In [ ]:
print(concat_ctd.shape)

In [ ]:
## Save p_features
ctd_sparse = sp.csr_matrix(concat_ctd, dtype=float)
print(ctd_sparse)

In [ ]:
p_save_path = data_path + 'protein_embeddings/'
sparse.save_npz(p_save_path + 'ctd.npz', ctd_sparse)